# 🏭 Bosch 생산라인 성능 분석 (한글 학습용)

## 📚 학습 목표
이 노트북을 통해 여러분은 다음을 배우게 됩니다:
1. **빅데이터 처리**: 대용량 데이터를 효율적으로 다루는 방법
2. **머신러닝 기초**: 예측 모델을 만드는 과정
3. **불균형 데이터**: 희귀한 사건을 예측하는 방법
4. **실무 프로젝트**: 실제 제조업 문제 해결

## 🎯 프로젝트 개요

### 배경 설명
- **Bosch**: 독일의 유명한 자동차 부품 제조 회사
- **문제**: 생산라인에서 불량품을 미리 예측하고 싶음
- **목표**: 제품이 불량품인지 정상품인지 예측하기

### 왜 이게 중요한가요?
- 💰 **비용 절감**: 불량품을 미리 발견하면 비용을 아낄 수 있음
- ⏱️ **시간 단축**: 문제를 빨리 발견하면 생산 효율이 올라감
- 📈 **품질 향상**: 불량 원인을 파악해 품질을 개선할 수 있음

### 데이터 특징
- **매우 큰 데이터**: 수천 개의 측정값 (특징)
- **불균형한 데이터**: 불량품이 전체의 1% 미만 (매우 희귀!)
- **세 가지 데이터 유형**: 숫자, 범주형, 날짜
- **익명화된 데이터**: 보안상 실제 측정 항목명은 숨김

## 1. 📦 필요한 도구 가져오기 (라이브러리 임포트)

프로그래밍에서는 이미 만들어진 도구들을 가져와서 사용합니다.
마치 요리할 때 필요한 도구를 꺼내놓는 것과 같아요!

In [ ]:
# 데이터 처리 도구들
import pandas as pd          # 엑셀같은 표 형태의 데이터를 다루는 도구
import numpy as np           # 수학 계산을 빠르게 해주는 도구

# 시각화 도구들 (그래프 그리기)
import matplotlib.pyplot as plt  # 기본적인 그래프를 그리는 도구
import seaborn as sns           # 예쁜 그래프를 그리는 도구

# 파일 처리 도구들
from zipfile import ZipFile     # 압축파일을 푸는 도구
import os                       # 컴퓨터의 파일을 관리하는 도구

# 머신러닝 도구들
from sklearn.model_selection import train_test_split     # 데이터를 나누는 도구
from sklearn.preprocessing import StandardScaler        # 데이터 크기를 맞추는 도구
from sklearn.metrics import matthews_corrcoef          # 모델 성능을 측정하는 도구

# 메모리 관리
import gc                       # 메모리를 청소하는 도구 (쓰레기 수집기)

# 경고 메시지 숨기기 (깔끔한 출력을 위해)
import warnings
warnings.filterwarnings('ignore')

# 그래프 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')  # 예쁜 그래프 스타일 적용
%matplotlib inline                       # 노트북 안에 그래프 표시

## 2. 📂 데이터 불러오기

### 💡 학습 포인트
- 대용량 데이터는 압축되어 있는 경우가 많음
- 압축을 풀고 데이터를 읽는 과정이 필요

In [ ]:
# 데이터가 저장된 폴더 지정
data_dir = './data/'  # 현재 위치의 data 폴더를 의미

# 폴더 안에 어떤 파일들이 있는지 확인
data_files = os.listdir(data_dir)  # 폴더 안의 파일 목록 가져오기
print("📁 데이터 폴더에 있는 파일들:")
print("=" * 30)
for file in data_files:
    # 파일 크기도 함께 표시
    file_path = os.path.join(data_dir, file)
    if os.path.isfile(file_path):
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"  📄 {file} ({size_mb:.2f} MB)")

In [ ]:
def extract_zip_files(data_dir):
    """
    압축 파일을 자동으로 푸는 함수
    
    왜 함수를 만드나요?
    - 같은 작업을 반복할 때 편리
    - 코드를 깔끔하게 정리
    - 나중에 다시 사용 가능
    """
    # .zip으로 끝나는 파일들만 찾기
    zip_files = [f for f in os.listdir(data_dir) if f.endswith('.zip')]
    
    print(f"\n🔍 발견된 압축 파일: {len(zip_files)}개")
    
    for zip_file in zip_files:
        file_path = os.path.join(data_dir, zip_file)
        csv_file = zip_file.replace('.zip', '')  # .zip을 제거한 이름
        csv_path = os.path.join(data_dir, csv_file)
        
        # 이미 압축이 풀려있는지 확인
        if not os.path.exists(csv_path):
            print(f"\n📦 압축 해제 중: {zip_file}...")
            with ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(data_dir)  # 압축 풀기
            print(f"   ✅ 완료! → {csv_file}")
        else:
            print(f"   ⏭️  {csv_file} 이미 존재 (건너뜀)")

# 함수 실행
extract_zip_files(data_dir)

## 3. 🔍 데이터 탐색하기

### 💡 왜 샘플만 먼저 볼까요?
- 전체 데이터가 너무 크면 컴퓨터가 느려질 수 있음
- 작은 샘플로도 데이터의 구조를 파악할 수 있음
- 테스트와 실험을 빠르게 할 수 있음

In [ ]:
# 숫자 데이터의 일부분만 읽어오기 (첫 10,000줄)
print("📊 train_numeric 데이터 샘플 불러오는 중...")
print("(전체 데이터가 크므로 일부만 읽어옵니다)\n")

# nrows=10000 : 처음 10,000줄만 읽기
train_numeric_sample = pd.read_csv(data_dir + 'train_numeric.csv', nrows=10000)

# 데이터의 크기 확인
print(f"📏 샘플 데이터 크기: {train_numeric_sample.shape}")
print(f"   → {train_numeric_sample.shape[0]:,}개의 제품 (행)")
print(f"   → {train_numeric_sample.shape[1]:,}개의 측정값 (열)")

# 처음 몇 개 열의 이름 보기
print(f"\n🏷️  처음 10개 컬럼명:")
for i, col in enumerate(train_numeric_sample.columns[:10], 1):
    print(f"   {i}. {col}")

In [ ]:
# 불량품 비율 확인하기
print("🎯 목표 변수(Response) 분포:")
print("=" * 40)

# Response = 0: 정상품, Response = 1: 불량품
response_counts = train_numeric_sample['Response'].value_counts()
print(f"정상품 (0): {response_counts[0]:,}개")
print(f"불량품 (1): {response_counts[1]:,}개")

# 불량률 계산
failure_rate = train_numeric_sample['Response'].mean()
print(f"\n⚠️  불량률: {failure_rate:.2%}")
print(f"   → 100개 중 약 {int(failure_rate * 100)}개가 불량품")

# 시각화
plt.figure(figsize=(8, 5))
train_numeric_sample['Response'].value_counts().plot(kind='bar', 
                                                     color=['green', 'red'])
plt.title('정상품 vs 불량품 분포', fontsize=14, fontweight='bold')
plt.xlabel('제품 상태')
plt.ylabel('개수')
plt.xticks([0, 1], ['정상품(0)', '불량품(1)'], rotation=0)
plt.grid(axis='y', alpha=0.3)

# 각 막대 위에 숫자 표시
for i, v in enumerate(response_counts):
    plt.text(i, v + 50, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💭 이것이 의미하는 것:")
print("   • 매우 불균형한 데이터 (불량품이 극히 적음)")
print("   • 특별한 처리 방법이 필요함")

In [ ]:
# 데이터 기본 정보 확인
print("📋 데이터셋 기본 정보:")
print("=" * 40)

# Id와 Response를 제외한 실제 특징의 개수
num_features = len(train_numeric_sample.columns) - 2
num_samples = len(train_numeric_sample)

print(f"🔢 측정 항목 수: {num_features:,}개")
print(f"📦 샘플 수: {num_samples:,}개")

# 결측값 (빈 값) 비율 계산
print(f"\n❓ 결측값(빈 데이터) 분석:")
print("-" * 40)

# 각 컬럼별 결측값 비율 계산
missing_percent = (train_numeric_sample.isnull().sum() / len(train_numeric_sample) * 100)
missing_percent = missing_percent.sort_values(ascending=False)

# 결측값이 많은 상위 10개 컬럼
print("결측값이 가장 많은 10개 컬럼:")
for col, pct in missing_percent.head(10).items():
    print(f"   • {col}: {pct:.1f}% 비어있음")

# 전체 결측값 통계
print(f"\n📊 결측값 통계:")
print(f"   • 완전히 비어있는 컬럼: {(missing_percent == 100).sum()}개")
print(f"   • 50% 이상 비어있는 컬럼: {(missing_percent >= 50).sum()}개")
print(f"   • 결측값이 전혀 없는 컬럼: {(missing_percent == 0).sum()}개")

## 4. 📊 특징(Feature) 분석하기

### 💡 학습 포인트
- 특징(Feature): 머신러닝에서 예측에 사용하는 각각의 측정값
- 특징이 많다고 무조건 좋은 것은 아님
- 중요한 특징을 찾아내는 것이 핵심!

In [ ]:
def analyze_feature_groups(df):
    """
    특징들을 그룹별로 분류하는 함수
    
    예: L0_S0_F0, L0_S0_F2 → 'L0_S0' 그룹
    이렇게 하면 어떤 생산 라인/스테이션에 특징이 많은지 알 수 있음
    """
    feature_groups = {}  # 빈 사전(dictionary) 생성
    
    # 각 컬럼을 하나씩 확인
    for col in df.columns:
        if col not in ['Id', 'Response']:  # Id와 Response는 제외
            # 언더바(_)를 기준으로 분리
            parts = col.split('_')
            if len(parts) >= 2:
                # 처음 두 부분을 그룹명으로 사용
                group = parts[0] + '_' + parts[1]
                
                # 그룹이 없으면 새로 만들기
                if group not in feature_groups:
                    feature_groups[group] = []
                
                # 그룹에 특징 추가
                feature_groups[group].append(col)
    
    # 특징이 많은 순서대로 정렬
    sorted_groups = sorted(feature_groups.items(), 
                          key=lambda x: len(x[1]), 
                          reverse=True)
    
    print("🏭 생산 라인/스테이션별 특징 개수 (상위 10개):")
    print("=" * 50)
    for i, (group, features) in enumerate(sorted_groups[:10], 1):
        print(f"{i:2d}. {group}: {len(features):3d}개의 측정값")
        # 막대 그래프 형태로 시각화
        bar_length = int(len(features) / 5)  # 5개당 1개 막대
        print(f"    {'█' * bar_length}")
    
    return feature_groups

# 함수 실행
feature_groups = analyze_feature_groups(train_numeric_sample)

print("\n💭 해석:")
print("   • L은 Line(생산라인), S는 Station(작업장)을 의미할 가능성이 높음")
print("   • 각 라인/스테이션에서 여러 측정을 수행함")

In [ ]:
# 결측값 패턴을 시각화하기
plt.figure(figsize=(14, 6))

# 각 컬럼의 결측값 비율 계산
missing_df = pd.DataFrame({
    'column': train_numeric_sample.columns,
    'missing_percent': (train_numeric_sample.isnull().sum() / len(train_numeric_sample) * 100)
})

# 그래프 1: 결측값 비율의 분포 (히스토그램)
plt.subplot(1, 2, 1)
plt.hist(missing_df['missing_percent'], bins=50, edgecolor='black', alpha=0.7, color='skyblue')
plt.xlabel('결측값 비율 (%)', fontsize=11)
plt.ylabel('특징 개수', fontsize=11)
plt.title('📊 특징들의 결측값 분포', fontsize=13, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

# 평균선 추가
mean_missing = missing_df['missing_percent'].mean()
plt.axvline(mean_missing, color='red', linestyle='--', linewidth=2, label=f'평균: {mean_missing:.1f}%')
plt.legend()

# 그래프 2: 누적 결측값 패턴
plt.subplot(1, 2, 2)
sorted_missing = np.sort(missing_df['missing_percent'].values)
plt.plot(range(len(sorted_missing)), sorted_missing, linewidth=2, color='darkblue')
plt.xlabel('특징 순서 (결측값 적은 순)', fontsize=11)
plt.ylabel('결측값 비율 (%)', fontsize=11)
plt.title('📈 누적 결측값 패턴', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)

# 50% 선 표시
plt.axhline(50, color='orange', linestyle='--', linewidth=1, label='50% 기준선')
plt.legend()

plt.tight_layout()
plt.show()

print("\n💡 결측값 분석 인사이트:")
print("   • 많은 특징들이 대부분 비어있음 (90% 이상)")
print("   • 이는 모든 제품이 모든 검사를 받지 않음을 의미")
print("   • 결측값 자체도 중요한 정보일 수 있음 (특정 검사를 건너뛴 이유?)")

## 5. 💾 메모리 효율적으로 사용하기

### 💡 왜 메모리 최적화가 중요한가요?
- 컴퓨터 메모리는 한정되어 있음
- 대용량 데이터를 다룰 때는 메모리 관리가 필수
- 메모리를 효율적으로 쓰면 더 많은 데이터를 처리 가능

In [ ]:
def reduce_memory_usage(df):
    """
    데이터 타입을 최적화하여 메모리 사용량을 줄이는 함수
    
    예시: 
    - 0과 1만 있는 데이터 → int64 대신 int8 사용 (8배 절약!)
    - 작은 숫자만 있는 데이터 → float64 대신 float32 사용 (2배 절약!)
    """
    # 현재 메모리 사용량 계산
    start_mem = df.memory_usage().sum() / 1024**2  # MB 단위로 변환
    print(f'💾 현재 메모리 사용량: {start_mem:.2f} MB')
    print('\n최적화 진행 중...')
    
    # 각 컬럼을 하나씩 최적화
    for col in df.columns:
        col_type = df[col].dtype  # 현재 데이터 타입
        
        if col_type != object:  # 숫자형 데이터만 처리
            c_min = df[col].min()  # 최솟값
            c_max = df[col].max()  # 최댓값
            
            # 정수형 데이터 최적화
            if str(col_type)[:3] == 'int':
                # 값의 범위에 따라 가장 작은 타입 선택
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)  # -128 ~ 127
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)  # -32768 ~ 32767
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            
            # 실수형 데이터 최적화
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
    
    # 최적화 후 메모리 사용량
    end_mem = df.memory_usage().sum() / 1024**2
    
    # 결과 출력
    print(f'\n✅ 최적화 완료!')
    print(f'💾 최적화 후 메모리: {end_mem:.2f} MB')
    print(f'📉 절약된 메모리: {start_mem - end_mem:.2f} MB')
    print(f'🎯 감소율: {100 * (start_mem - end_mem) / start_mem:.1f}%')
    
    return df

# 메모리 최적화 실행
print("🚀 메모리 최적화를 시작합니다...\n")
train_numeric_sample = reduce_memory_usage(train_numeric_sample)

print("\n💡 메모리 최적화의 효과:")
print("   • 같은 데이터를 더 작은 공간에 저장")
print("   • 더 빠른 처리 속도")
print("   • 더 많은 데이터를 한번에 처리 가능")

## 6. 🛠️ 특징 공학 (Feature Engineering)

### 💡 특징 공학이란?
- 기존 데이터를 가공해서 새로운 유용한 특징을 만드는 것
- 예: 키와 몸무게 → BMI 지수 계산
- 좋은 특징을 만들면 예측 성능이 크게 향상됨!

In [ ]:
def create_basic_features(df):
    """
    기본적인 통계 특징들을 생성하는 함수
    
    각 제품(행)에 대해 전체 측정값들의 통계를 계산
    이렇게 하면 수천 개의 특징을 몇 개의 중요한 특징으로 요약 가능
    """
    print("🔧 새로운 특징을 만드는 중...\n")
    
    feature_df = pd.DataFrame()  # 새로운 데이터프레임 생성
    feature_df['Id'] = df['Id']  # Id 복사
    
    # 1. 비어있지 않은 측정값의 개수
    # (얼마나 많은 검사를 받았는지)
    feature_df['count_non_null'] = df.drop(['Id', 'Response'], axis=1, errors='ignore').count(axis=1)
    print("✅ 측정값 개수 계산 완료")
    
    # 2. 0인 값의 개수
    # (측정값이 0이라는 것도 의미가 있을 수 있음)
    feature_df['count_zeros'] = (df.drop(['Id', 'Response'], axis=1, errors='ignore') == 0).sum(axis=1)
    print("✅ 0값 개수 계산 완료")
    
    # 3. 기본 통계량 계산 (각 제품별로)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.drop(['Id', 'Response'], errors='ignore')
    
    feature_df['mean'] = df[numeric_cols].mean(axis=1)      # 평균
    feature_df['std'] = df[numeric_cols].std(axis=1)        # 표준편차 (데이터의 흩어진 정도)
    feature_df['min'] = df[numeric_cols].min(axis=1)        # 최솟값
    feature_df['max'] = df[numeric_cols].max(axis=1)        # 최댓값
    feature_df['median'] = df[numeric_cols].median(axis=1)  # 중앙값
    print("✅ 통계량 계산 완료")
    
    # 4. 결측값 비율
    # (얼마나 많은 검사를 건너뛰었는지)
    feature_df['missing_percent'] = df[numeric_cols].isnull().sum(axis=1) / len(numeric_cols)
    print("✅ 결측값 비율 계산 완료")
    
    # Response (목표 변수) 추가
    if 'Response' in df.columns:
        feature_df['Response'] = df['Response']
    
    return feature_df

# 특징 생성 실행
engineered_features = create_basic_features(train_numeric_sample)

print(f"\n📊 생성된 특징 요약:")
print(f"   원래 특징 개수: {len(train_numeric_sample.columns)}개")
print(f"   새로운 특징 개수: {len(engineered_features.columns)}개")
print(f"\n📝 새로 만든 특징들:")
for i, col in enumerate(engineered_features.columns, 1):
    if col not in ['Id', 'Response']:
        print(f"   {i-1}. {col}")

# 새로운 특징들의 예시 보기
print("\n🔍 처음 5개 제품의 새로운 특징값:")
engineered_features.head()

In [ ]:
# 새로 만든 특징들이 불량품 예측에 얼마나 유용한지 확인
if 'Response' in engineered_features.columns:
    # 각 특징과 목표 변수의 상관관계 계산
    correlations = engineered_features.drop(['Id', 'Response'], axis=1).corrwith(engineered_features['Response'])
    correlations = correlations.sort_values(ascending=False)
    
    # 시각화
    plt.figure(figsize=(12, 6))
    colors = ['green' if x > 0 else 'red' for x in correlations]
    correlations.plot(kind='bar', color=colors)
    plt.title('🎯 새로운 특징들과 불량품의 상관관계', fontsize=14, fontweight='bold')
    plt.xlabel('특징', fontsize=12)
    plt.ylabel('상관계수', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.axhline(0, color='black', linewidth=0.5)
    
    # 가장 중요한 값에 주석 추가
    max_corr = correlations.abs().max()
    max_feature = correlations.abs().idxmax()
    plt.annotate(f'가장 높은 상관관계\n{max_feature}', 
                xy=(list(correlations.index).index(max_feature), correlations[max_feature]),
                xytext=(0, 0.05),
                fontsize=10,
                ha='center',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.5),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 상관관계 분석 결과:")
    print("=" * 50)
    for feature, corr in correlations.items():
        if abs(corr) > 0.01:  # 의미있는 상관관계만 표시
            direction = "양의" if corr > 0 else "음의"
            print(f"• {feature:20s}: {corr:+.4f} ({direction} 상관관계)")
    
    print("\n💡 해석 방법:")
    print("   • 양의 상관관계 (+): 값이 클수록 불량품일 가능성 높음")
    print("   • 음의 상관관계 (-): 값이 작을수록 불량품일 가능성 높음")
    print("   • 0에 가까움: 불량품 예측과 관련이 적음")

## 7. 📦 대용량 데이터 처리 전략

### 💡 청크(Chunk) 처리란?
- 큰 파일을 작은 조각으로 나누어 처리
- 예: 책 전체를 한번에 읽기 vs 한 장씩 읽기
- 메모리 부족 문제를 해결하는 핵심 기술

In [ ]:
def process_chunk(chunk, feature_list=None):
    """
    데이터 청크(조각)를 처리하는 함수
    
    큰 파일을 작은 조각으로 나누어 처리할 때 사용
    각 조각을 처리하고 결과를 합치면 전체 처리 완료!
    """
    # 특정 컬럼만 선택 (메모리 절약)
    if feature_list is not None:
        available_cols = [col for col in feature_list if col in chunk.columns]
        chunk = chunk[available_cols + ['Id', 'Response']]
    
    # 메모리 최적화
    chunk = reduce_memory_usage(chunk)
    
    # 특징 생성
    features = create_basic_features(chunk)
    
    return features

print("📚 청크 처리 예시 코드:")
print("=" * 50)
print("""
# 실제로 대용량 파일을 처리할 때 사용하는 코드
chunk_size = 50000  # 한 번에 50,000줄씩 읽기
chunks = []  # 처리된 조각들을 저장할 리스트

# 파일을 조각별로 읽어서 처리
for chunk in pd.read_csv('big_file.csv', chunksize=chunk_size):
    processed_chunk = process_chunk(chunk)  # 조각 처리
    chunks.append(processed_chunk)          # 결과 저장
    gc.collect()                            # 메모리 청소

# 모든 조각을 합치기
full_features = pd.concat(chunks, ignore_index=True)
""")

print("\n💡 청크 처리의 장점:")
print("   1. 메모리 부족 문제 해결")
print("   2. 진행 상황을 확인 가능")
print("   3. 중간에 멈춰도 일부 결과는 저장됨")
print("   4. 병렬 처리 가능")

## 8. 🎯 중요한 특징 선택하기

### 💡 왜 특징 선택이 중요한가요?
- 모든 특징이 다 유용한 것은 아님
- 쓸모없는 특징은 오히려 성능을 떨어뜨림
- 적은 특징으로도 좋은 예측이 가능

In [ ]:
def select_important_features(df, target_col='Response', threshold=0.95):
    """
    중요한 특징만 선택하는 함수
    
    선택 기준:
    1. 결측값이 너무 많지 않은 특징 (95% 미만)
    2. 변화가 있는 특징 (분산이 0이 아님)
    """
    print(f"🔍 특징 선택을 시작합니다...")
    print(f"   초기 특징 개수: {len(df.columns)}개\n")
    
    # 1단계: 결측값이 너무 많은 특징 제거
    missing_percent = df.isnull().sum() / len(df)
    keep_cols = missing_percent[missing_percent < threshold].index.tolist()
    
    removed_by_missing = len(df.columns) - len(keep_cols)
    print(f"📉 결측값 필터 적용:")
    print(f"   • 제거된 특징: {removed_by_missing}개")
    print(f"   • 남은 특징: {len(keep_cols)}개")
    
    # 2단계: 변화가 없는 특징 제거 (분산이 매우 작은 특징)
    numeric_cols = df[keep_cols].select_dtypes(include=[np.number]).columns
    variances = df[numeric_cols].var()
    keep_cols = variances[variances > 0.01].index.tolist()
    
    removed_by_variance = len(numeric_cols) - len(keep_cols)
    print(f"\n📉 분산 필터 적용:")
    print(f"   • 제거된 특징: {removed_by_variance}개")
    print(f"   • 남은 특징: {len(keep_cols)}개")
    
    # Id와 Response는 항상 포함
    if 'Id' not in keep_cols:
        keep_cols.append('Id')
    if target_col in df.columns and target_col not in keep_cols:
        keep_cols.append(target_col)
    
    # 감소율 계산
    reduction_rate = (1 - len(keep_cols) / len(df.columns)) * 100
    
    print(f"\n✅ 특징 선택 완료!")
    print(f"   최종 특징 개수: {len(keep_cols)}개")
    print(f"   전체 감소율: {reduction_rate:.1f}%")
    
    return keep_cols

# 특징 선택 실행
important_features = select_important_features(train_numeric_sample)

print("\n💡 특징 선택의 효과:")
print("   • 학습 시간 단축")
print("   • 과적합(overfitting) 방지")
print("   • 모델 해석이 쉬워짐")
print("   • 메모리 사용량 감소")

## 9. 🤖 머신러닝 모델 학습

### 💡 머신러닝이란?
- 컴퓨터가 데이터를 보고 스스로 패턴을 학습
- 학습한 패턴을 바탕으로 새로운 데이터를 예측
- 예: 스팸 메일 필터, 추천 시스템 등

In [ ]:
# 필요한 머신러닝 라이브러리 임포트
print("🤖 머신러닝 라이브러리를 불러오는 중...\n")

from sklearn.ensemble import RandomForestClassifier  # 랜덤 포레스트 (숲)
from sklearn.linear_model import LogisticRegression  # 로지스틱 회귀
from xgboost import XGBClassifier                    # XGBoost (강력한 모델)
from sklearn.model_selection import StratifiedKFold  # 교차 검증
from imblearn.over_sampling import SMOTE            # 오버샘플링
from imblearn.under_sampling import RandomUnderSampler  # 언더샘플링

print("✅ 모델 라이브러리 준비 완료!")
print("\n📚 사용할 모델들:")
print("   1. Random Forest: 여러 개의 결정 트리를 합친 모델")
print("   2. Logistic Regression: 확률을 예측하는 기본 모델")
print("   3. XGBoost: 경진대회에서 자주 우승하는 강력한 모델")

In [ ]:
def prepare_data_for_modeling(df, target_col='Response'):
    """
    모델 학습을 위해 데이터를 준비하는 함수
    
    단계:
    1. X(입력)와 y(출력) 분리
    2. 결측값 처리
    3. 스케일링 (모든 특징의 크기를 비슷하게 맞춤)
    """
    print("🔧 모델링을 위한 데이터 준비 중...\n")
    
    # X(특징)와 y(목표) 분리
    X = df.drop(['Id', target_col], axis=1, errors='ignore')
    y = df[target_col] if target_col in df.columns else None
    
    print(f"📊 데이터 분리:")
    print(f"   • X (입력 특징): {X.shape}")
    print(f"   • y (목표 변수): {y.shape if y is not None else 'None'}")
    
    # 결측값을 중앙값으로 채우기
    X = X.fillna(X.median())
    print(f"\n✅ 결측값 처리 완료 (중앙값으로 대체)")
    
    # 스케일링 (표준화)
    # 모든 특징의 평균을 0, 표준편차를 1로 맞춤
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    print(f"✅ 스케일링 완료 (평균=0, 표준편차=1)")
    
    return X_scaled, y, scaler

# 데이터 준비 실행
X_sample, y_sample, scaler = prepare_data_for_modeling(engineered_features)

print(f"\n🎯 준비된 데이터:")
print(f"   • 학습 데이터 크기: {X_sample.shape}")
print(f"   • 특징 개수: {X_sample.shape[1]}개")
print(f"   • 샘플 개수: {X_sample.shape[0]}개")

In [ ]:
def handle_imbalance(X, y, strategy='undersample', ratio=0.1):
    """
    불균형한 데이터를 처리하는 함수
    
    문제: 불량품이 너무 적음 (1% 미만)
    해결법:
    1. 오버샘플링: 소수 클래스(불량품)를 인위적으로 늘림
    2. 언더샘플링: 다수 클래스(정상품)를 줄임
    """
    print(f"⚖️  불균형 처리 전략: {strategy}\n")
    
    # 현재 클래스 분포 확인
    print("현재 클래스 분포:")
    print(f"   • 정상품: {(y == 0).sum():,}개")
    print(f"   • 불량품: {(y == 1).sum():,}개")
    print(f"   • 불량률: {y.mean():.2%}\n")
    
    if strategy == 'oversample':
        # SMOTE: 소수 클래스의 합성 샘플을 생성
        sampler = SMOTE(sampling_strategy=ratio, random_state=42)
        print("📈 오버샘플링: 불량품 데이터를 인위적으로 생성")
    elif strategy == 'undersample':
        # 언더샘플링: 다수 클래스를 무작위로 제거
        sampler = RandomUnderSampler(sampling_strategy=ratio, random_state=42)
        print("📉 언더샘플링: 정상품 데이터를 일부 제거")
    else:
        return X, y
    
    # 리샘플링 실행
    X_resampled, y_resampled = sampler.fit_resample(X, y)
    
    print(f"\n✅ 리샘플링 완료!")
    print(f"   • 새로운 샘플 수: {len(y_resampled):,}개")
    print(f"   • 정상품: {(y_resampled == 0).sum():,}개")
    print(f"   • 불량품: {(y_resampled == 1).sum():,}개")
    print(f"   • 새로운 불량률: {y_resampled.mean():.2%}")
    
    return X_resampled, y_resampled

# 예시 (실제로는 실행하지 않음)
print("💡 불균형 처리 예시 코드:")
print("# X_balanced, y_balanced = handle_imbalance(X_sample, y_sample, strategy='undersample')")
print("\n📝 언제 어떤 방법을 사용하나요?")
print("   • 오버샘플링: 데이터가 적을 때")
print("   • 언더샘플링: 데이터가 충분히 많을 때")
print("   • 둘 다 사용: 적절히 조합하여 균형 맞추기")

## 10. 📝 학습 내용 정리

### 🎯 핵심 개념 정리

#### 1. 데이터 과학의 전체 프로세스
```
데이터 수집 → 전처리 → 탐색 → 특징 공학 → 모델링 → 평가 → 배포
```

#### 2. 이 프로젝트에서 배운 주요 기술
- **대용량 데이터 처리**: 청크 단위 처리, 메모리 최적화
- **불균형 데이터 처리**: SMOTE, 언더샘플링
- **특징 공학**: 통계적 특징 생성
- **머신러닝**: 분류 모델 학습

#### 3. 실무에서 중요한 포인트
- 데이터의 품질이 모델 성능을 좌우
- 도메인 지식이 중요 (제조업 이해)
- 평가 지표 선택이 중요 (MCC vs Accuracy)

### 💼 실제 적용 예시
이런 기술들은 다음과 같은 곳에서 사용됩니다:
- 🏭 제조업: 품질 관리, 불량 예측
- 🏥 의료: 질병 진단, 위험 예측
- 💰 금융: 사기 탐지, 신용 평가
- 🛒 이커머스: 추천 시스템, 고객 이탈 예측

## 11. 🚀 전체 파이프라인 템플릿

### 실제 프로젝트에서 사용할 수 있는 완전한 코드

In [ ]:
def full_pipeline():
    """
    Bosch 데이터셋을 위한 완전한 처리 파이프라인
    
    이 함수 하나로 전체 프로세스를 실행할 수 있습니다!
    """
    print("🚀 전체 파이프라인 시작!\n")
    print("=" * 60)
    
    # 1단계: 파일 압축 해제
    print("\n📦 1단계: 파일 준비")
    extract_zip_files(data_dir)
    
    # 2단계: 청크 단위로 데이터 처리
    print("\n📊 2단계: 데이터 처리 (청크 단위)")
    chunk_size = 50000
    numeric_features = []
    
    # 진행 상황을 보여주기 위한 카운터
    chunk_count = 0
    
    for chunk in pd.read_csv(data_dir + 'train_numeric.csv', chunksize=chunk_size):
        chunk_count += 1
        print(f"   처리 중: 청크 #{chunk_count}")
        
        # 메모리 최적화
        chunk = reduce_memory_usage(chunk)
        
        # 특징 생성
        features = create_basic_features(chunk)
        numeric_features.append(features)
        
        # 메모리 정리
        gc.collect()
    
    # 3단계: 모든 청크 합치기
    print("\n🔗 3단계: 데이터 통합")
    full_features = pd.concat(numeric_features, ignore_index=True)
    print(f"   통합된 데이터 크기: {full_features.shape}")
    
    # 4단계: 모델 학습 준비
    print("\n🔧 4단계: 모델 학습 준비")
    X, y, scaler = prepare_data_for_modeling(full_features)
    
    # 5단계: 교차 검증 설정
    print("\n📈 5단계: 교차 검증 설정")
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    print("   5-폴드 교차 검증 준비 완료")
    
    # 6단계: XGBoost 모델 생성
    print("\n🤖 6단계: XGBoost 모델 생성")
    model = XGBClassifier(
        n_estimators=100,        # 트리 개수
        max_depth=5,            # 트리 깊이
        learning_rate=0.1,      # 학습률
        scale_pos_weight=100,   # 불균형 처리
        random_state=42
    )
    print("   모델 설정 완료")
    
    # 7단계: 모델 학습 (예시)
    print("\n📚 7단계: 모델 학습")
    print("   (실제 학습은 시간이 오래 걸림)")
    # model.fit(X_train, y_train)
    
    print("\n✅ 파이프라인 완료!")
    print("=" * 60)
    
    return model

# 파이프라인 정의 완료
print("📋 전체 파이프라인 템플릿이 준비되었습니다!")
print("\n실행하려면:")
print(">>> model = full_pipeline()")
print("\n⚠️  주의: 전체 데이터 처리는 시간이 오래 걸립니다!")

## 12. 📤 제출 파일 만들기

### Kaggle 대회에 제출할 예측 결과 생성

In [ ]:
# 제출 템플릿 파일 확인
print("📄 제출 템플릿 파일 불러오기...\n")

submission = pd.read_csv(data_dir + 'sample_submission.csv')
print(f"📏 제출 파일 크기: {submission.shape}")
print(f"🏷️  컬럼: {submission.columns.tolist()}\n")

print("👀 제출 파일 미리보기:")
print(submission.head())
print("\n📝 설명:")
print("   • Id: 테스트 데이터의 각 제품 ID")
print("   • Response: 우리가 예측해야 할 값 (불량품 확률)")

In [ ]:
def create_submission(model, test_features, submission_template):
    """
    제출 파일을 생성하는 함수
    
    단계:
    1. 테스트 데이터로 예측
    2. 예측 결과를 제출 형식에 맞게 정리
    3. CSV 파일로 저장
    """
    print("📤 제출 파일 생성 중...\n")
    
    # 예측 수행 (불량품일 확률)
    predictions = model.predict_proba(test_features)[:, 1]
    print(f"✅ 예측 완료: {len(predictions):,}개 제품")
    
    # 제출 데이터프레임 생성
    submission = submission_template.copy()
    submission['Response'] = predictions
    
    # 예측 결과 통계
    print(f"\n📊 예측 결과 통계:")
    print(f"   • 평균 불량 확률: {predictions.mean():.4f}")
    print(f"   • 최소값: {predictions.min():.4f}")
    print(f"   • 최대값: {predictions.max():.4f}")
    print(f"   • 0.5 이상 (불량품 예측): {(predictions >= 0.5).sum():,}개")
    
    # 파일 저장
    submission.to_csv('submission.csv', index=False)
    print(f"\n💾 제출 파일 저장 완료: submission.csv")
    
    return submission

print("✅ 제출 함수 준비 완료!")
print("\n사용 예시:")
print(">>> submission = create_submission(model, X_test, submission_template)")

## 🎓 최종 요약 및 다음 단계

### 🏆 축하합니다! 
여러분은 실제 Kaggle 대회 데이터로 머신러닝 프로젝트를 완성했습니다!

### 📚 오늘 배운 핵심 내용
1. **빅데이터 처리**: 청크 처리, 메모리 최적화
2. **탐색적 데이터 분석**: 데이터 이해와 시각화
3. **특징 공학**: 새로운 유용한 특징 만들기
4. **불균형 데이터**: 희귀 이벤트 예측 방법
5. **머신러닝 파이프라인**: 전체 프로세스 구축

### 🚀 다음 단계 추천
1. **모델 개선하기**
   - 다른 알고리즘 시도 (LightGBM, CatBoost)
   - 하이퍼파라미터 튜닝
   - 앙상블 기법 적용

2. **더 깊이 배우기**
   - 날짜 데이터 활용
   - 범주형 데이터 처리
   - 고급 특징 공학

3. **다른 프로젝트 도전**
   - Kaggle의 다른 대회 참가
   - 자신만의 데이터 프로젝트 시작

### 💪 격려의 말
데이터 과학은 실습이 가장 중요합니다. 
실수를 두려워하지 말고 계속 도전하세요!

**"The expert in anything was once a beginner."**

### 📧 질문이나 피드백
이 노트북에 대한 질문이나 개선 사항이 있다면 언제든 문의해주세요!

---
**Happy Learning! 🎉**